# Задание

Построить модель машинного обучения, позволяющую предсказать, является ли цветок цветком класса Ирис-Вирджиния или нет.

## Шаг 1
Из библиотеки scikit-learn подгрузите данные. Мы уже работали с ними в рамках Темы 1.

In [49]:
from sklearn.datasets import load_iris
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
from scipy import stats

Изучим датасет

In [50]:
df = load_iris(as_frame=True).frame

In [51]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


In [53]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


## Шаг 2
Сформируйте новое поле target_virginica, которое будет равно из 0, если цветок не является классом 2, иначе  — 1. Убедитесь, что разметка выполнена корректно, рассчитав количество объектов каждого класса по новой целевой переменной target_virginica.

Перезапишем новый датасет, с помощью которого можно будет обучать модели.

In [54]:
iris_data = load_iris()
df = pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
df['target'] = iris_data.target
df['target_virginica'] = df['target'].apply(lambda x: 0 if x != 2 else 1)
count_by_class = df['target_virginica'].value_counts()
display(count_by_class)


target_virginica
0    100
1     50
Name: count, dtype: int64

## Шаг 3
Обучите библиотечную модель логистической регрессии на обучающей части данных.

In [55]:
X = df
y = df['target_virginica']
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Создание и обучение модели логистической регрессии
model = LogisticRegression()
model.fit(X_train, y_train)
# Оценка точности модели на тестовой выборке
accuracy = model.score(X_test, y_test)
display('Accuracy:', accuracy)

'Accuracy:'

1.0

## Шаг 4
Оцените качество обученной на предыдущем шаге модели, выведя матрицу ошибок для прогнозов обученной моделью на тестовой части данных.

In [56]:
# Прогнозы модели на тестовой выборке
y_pred = model.predict(X_test)
# Вычисление матрицы ошибок
confusion_mat = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(confusion_mat)

Confusion Matrix:
[[19  0]
 [ 0 11]]


*Вывод:* Все классы были точно предсказаны для всех примеров тестовой выборки

## Шаг 5
Сконструируйте свой класс MyLogisticRegression, решающий задачу логистической регрессии методом градиентного спуска.
- Он должен инициализироваться величиной шага градиентного спуска и количеством итераций градиентного спуска.
- Должен содержать метод fit для обучения модели, на вход которой будут подаваться обучающие данные. Чтобы реализовать функцию sigmoid, можно воспользоваться библиотечной функцией np.exp. Для расчета градиента необходимо использовать формулу, приведенную в курсе в блоке «Логистическая регрессия для решения задачи бинарной классификации».
- Должен содержаться метод predict, который будет возвращать лейбл 0/1. Лейбл должен рассчитываться из вероятности принадлежности классу 1. Вероятность должна рассчитываться как сигмоида от результата произведения матрицы фичей X и вектора весов модели w.

In [57]:
class MyLogisticRegression:
    def __init__(self, learning_rate=0.01, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None

    def sigmoid(self, z):
            return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        # Добавление столбца единиц в матрицу X для учета bias
        X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

        # Инициализация весов случайным образом
        self.weights = np.random.rand(X.shape[1])

        # Градиентный спуск
        for _ in range(self.num_iterations):
            # Расчет предсказаний
            y_pred = self.sigmoid(np.dot(X, self.weights))

            # Расчет градиента
            gradient = np.dot(X.T, y_pred - y)

            # Обновление весов
            self.weights -= self.learning_rate * gradient

    def predict(self, X):
        # Добавление столбца единиц в марицу X для учета bias
        X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

        # Расчет вероятности принадлежности классу
        proba = self.sigmoid(np.dot(X, self.weights))

        # Преобразование вероятности в лейбл 0/1
        label = np.where(proba >= 0.5, 1, 0)
        return label



## Шаг 6
Обучите модель с помощью реализованного класса с произвольными параметрами градиентного спуска.

In [58]:
X = df
y = df['target_virginica']
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Создание и обучение модели логистической регрессии
model = MyLogisticRegression()
model.fit(X_train, y_train)

## Шаг 7
Оцените качество модели, обученной с помощью класса MyLogisticRegression, выведя матрицу ошибок для прогнозов обученной моделью на тестовой части данных.

In [59]:
y_pred = model.predict(X_test)
# Вычисление матрицы ошибок
confusion_mat = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(confusion_mat)

Confusion Matrix:
[[19  0]
 [ 0 11]]


## Шаг 8
Подберите такие параметры градиентного спуска, чтобы разделение получилось не хуже, чем у библиотечной модели.

learning_rate=0.01, num_iterations=1000 - данные параметры являются оптимальными и позволяют добиваться точности, не хуже, чем на библиотечной модели.

